## Q2: Air pollution (30 marks)

In this question, we will use the OpenWeather API to get the pollution data for different cities. Do the following:

1. Get a list of cities with their latitude and longitude information by doing the following:
    * (a) Download the list of cities from the table in https://en.wikipedia.org/wiki/List_of_cities_by_elevation using [`pd.read_html()`](https://pandas.pydata.org/docs/reference/api/pandas.read_html.html)
    * (b) Keep all the rows except those with Americas as Continental Region in a `pd.DataFrame`. Only keep columns  `City Name/s`, `Continental Region`, `Latitude` and `Longitude`. How many cities do you have in the `pd.DataFrame`?
    * (c) Convert the Latitude and Longitude from the use of "N", "E", "S", "W" to `float` with signs +, +, -, -
        
At the end you should get a `pd.DataFrame` like the following:
<img src="figs/cities.png" width="500"/>

Note only the first few rows are shown here.

2. Update the `key.json` file with your OpenWeather API key, read [ OpenWeather API price](https://openweathermap.org/price) to find out how many API calls you can make per minute, per day and per month with your free account, and understand how to use the Air Pollution API by reading the documentation [here](https://openweathermap.org/api/air-pollution)
    * You do not need to provide an answer for this part, but above are preparation works that will be useful when you call the API later 
3. Do the following:
    * (a) Make API calls using the [Air Pollution API](https://openweathermap.org/api/air-pollution) to collect the current air pollution data of the cities from (1) using the latitude and longitude information from (1)
    * (b) Combine the result from (1) and (3a) in a `pd.DataFrame` with the following 10 columns:
        * City name
        * Continental region
        * The concentration of 8 types of polluting gases and particulates (8 separate columns)
            * Carbon monoxide (CO), Nitrogen monoxide (NO), Nitrogen dioxide (NO2), Ozone (O3), Sulphur dioxide (SO2), Ammonia (NH3), and particulates (PM2.5 and PM10)
    * (c) Store the `pd.DataFrame` from (3b) into a csv file in the `data` folder, with the name of the file to be `air_pollutant_2022mmdd.csv` with `mm` the month and `dd` the day you have collected the data. This file will help us to verify your result

4. Use the `pd.DataFrame` from (3b) to investigate the relations between different variables by using some descriptive statistics like correlation and/or some aggregate functions:
    * If the concentration of one pollutant is high, is it more likely that the concentration of another pollutant is also high? 
    * Do you find any relations between continental region and the pollution levels?
    
    Please state the limitations of your answers.
    
---
### Hints
* For (1), you may find [`.apply()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) and/or [`.str.replace()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.replace.html) useful
* For (3), you may want to make use of [`.iterrows()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iterrows.html) to help you to iterate over each row

---

In [1]:
## your attempt, please add the code cells and markdown cells for your answers. Make sure you:
## * use the right type of cells
## * state clearly which answer is for which part
## * show the output of the code cells

# Q2.1

In [2]:
#(a)
import pandas as pd
from bs4 import BeautifulSoup 
import requests

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_cities_by_elevation'
page = requests.get(url)

In [4]:
df = pd.read_html( page.content )[1]
df.head()

,Country/Territory,City Name/s,Continental Region,Latitude,Longitude,Population,Elevation (m)
0,Nepal,Pokhara,Asia,N28.2096,E83.9856,523000.0,822
1,South Africa,Bloemfontein,Africa,S29.116667,E026.216667,747431.0,1395
2,China,Shanghai,Asia,N31.2304,E121.4737,26320000.0,122
3,Nepal,Butwal,Asia,N27.6866,E83.4323,120982.0,150
4,Italy,Milan,Europe,N45.4625,E9.186389,1378689.0,122


In [5]:
#(b)

In [6]:
# keep columns City Name/s, Continental Region, Latitude and Longitude
df_s = df[['City Name/s','Continental Region', 'Latitude', 'Longitude']]
df_s.head()

,City Name/s,Continental Region,Latitude,Longitude
0,Pokhara,Asia,N28.2096,E83.9856
1,Bloemfontein,Africa,S29.116667,E026.216667
2,Shanghai,Asia,N31.2304,E121.4737
3,Butwal,Asia,N27.6866,E83.4323
4,Milan,Europe,N45.4625,E9.186389


In [7]:
# remove the rows those with Americas 
df1 = df_s.drop(df_s[df_s['Continental Region'] == 'Americas'].index)
df1 = df1.reset_index(drop=True) # reset the index
df1.head()

,City Name/s,Continental Region,Latitude,Longitude
0,Pokhara,Asia,N28.2096,E83.9856
1,Bloemfontein,Africa,S29.116667,E026.216667
2,Shanghai,Asia,N31.2304,E121.4737
3,Butwal,Asia,N27.6866,E83.4323
4,Milan,Europe,N45.4625,E9.186389


In [8]:
#The num of cities
df1['City Name/s'].nunique()

159

 There are 159 cities in dataframe

In [9]:
#(c) 

In [10]:
df1['Latitude'] = df1['Latitude'].str.replace('N','+')
df1['Latitude'] = df1['Latitude'].str.replace('S','-')

In [11]:
df1['Longitude'] = df1['Longitude'].str.replace( 'E','+')
df1['Longitude'] = df1['Longitude'].str.replace( 'W','-')

In [13]:
#check
df1.head()

,City Name/s,Continental Region,Latitude,Longitude
0,Pokhara,Asia,+28.2096,+83.9856
1,Bloemfontein,Africa,-29.116667,+026.216667
2,Shanghai,Asia,+31.2304,+121.4737
3,Butwal,Asia,+27.6866,+83.4323
4,Milan,Europe,+45.4625,+9.186389


In [14]:
df1['Latitude'] = df1['Latitude'].astype(float)
df1['Longitude'] = df1['Longitude'].astype(float)

In [1]:
print(df1.dtypes)

NameError: name 'df1' is not defined

# Q2.2

In [16]:
from bs4 import BeautifulSoup
import requests
import json


In [17]:
#(a)
url = 'http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid=20b3992c11b8e46d6476b2c1ccc69939'
r = requests.get(url)

In [18]:
r

<Response [400]>

# Q2.3

In [19]:
#(a) 

In [20]:
import pandas as pd

In [21]:
result = []
air_po = []

for i in range(len(df1)):
    latitude = df1['Latitude'][i]
    longitude = df1['Longitude'][i]
    url = 'http://api.openweathermap.org/data/2.5/air_pollution?lat='+ str(latitude)+ '&lon='+str(longitude)+'&appid=20b3992c11b8e46d6476b2c1ccc69939'
    r1 = requests.get(url)
    p_data = json.loads(r1.text)
    air_po.append(p_data)

In [22]:
#(b)

In [23]:
air_po[0]['list'][0]['components']

{'co': 313.76,
 'no': 0,
 'no2': 1.41,
 'o3': 38.27,
 'so2': 0.57,
 'pm2_5': 18.35,
 'pm10': 29.11,
 'nh3': 2.69}

In [24]:
#collect the concentration of 8 types of polluting gases and particulates from the 
#  air pollution data(air_po list) that we got in 3(b)
co = []
no = []
no2 = []
o3 = []
so2 =[]
pm2_5 = []
pm10 = []
nh3 = []

for i in range(len(air_po)):
    
    co.append(air_po[i]['list'][0]['components']['co'])
    no.append(air_po[i]['list'][0]['components']['no'])
    no2 .append(air_po[i]['list'][0]['components']['no2'])
    o3.append(air_po[i]['list'][0]['components']['o3'])
    so2.append(air_po[i]['list'][0]['components']['so2'])
    pm2_5.append(air_po[i]['list'][0]['components']['pm2_5'])
    pm10.append(air_po[i]['list'][0]['components']['pm10'])
    nh3.append(air_po[i]['list'][0]['components']['nh3'])

In [25]:
# store the results in a pd.DataFrame air_df

In [26]:
# df of City name Continental region
air_df_CC = pd.DataFrame()
air_df_CC['City Name/s']= df1["City Name/s"]
air_df_CC['Continental Region']=df1['Continental Region']

In [27]:
# df of 8 types of polluting gases and particulates
zipped = zip(co,no,no2,o3,so2,pm2_5,pm10,nh3)
air_df_PP = pd.DataFrame(zipped, columns=['CO', 'NO', 'NO2', 'O3', 'SO2','PM2.5','PM10','NH3'])

In [28]:
air_df_full = pd.concat([air_df_CC,air_df_PP],axis=1)

In [29]:
#check
air_df_full.head()

,City Name/s,Continental Region,CO,NO,NO2,O3,SO2,PM2.5,PM10,NH3
0,Pokhara,Asia,313.76,0.00,1.41,38.27,0.57,18.35,29.11,2.69
1,Bloemfontein,Africa,223.64,0.00,4.41,26.11,6.50,4.59,4.98,0.00
2,Shanghai,Asia,423.91,8.05,31.87,75.82,42.44,48.82,80.47,0.03
3,Butwal,Asia,841.14,0.49,6.86,18.42,1.74,84.13,120.09,6.90
4,Milan,Europe,1348.50,36.66,37.01,0.00,1.43,56.42,58.60,1.77


In [30]:
#(c) Store the pd.DataFrame 
import os
air_df_full.to_csv('data/air_pollutant_2022_11_2.csv')

# Q2.4

In [31]:
air_df_full.describe()

,CO,NO,NO2,O3,SO2,PM2.5,PM10,NH3
count,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000
mean,591.168994,10.039497,14.709371,34.069182,9.620943,36.182830,67.432201,3.825912
std,982.632297,38.601769,19.203702,27.268131,21.104366,48.116774,106.360797,8.989483
min,165.220000,0.000000,0.070000,0.000000,0.060000,0.500000,0.520000,0.000000
25%,211.950000,0.000000,1.755000,12.335000,0.605000,4.570000,8.995000,0.310000
50%,307.080000,0.010000,6.860000,30.040000,2.060000,14.920000,23.700000,1.090000
75%,497.345000,0.300000,19.880000,50.425000,6.970000,48.065000,74.465000,2.610000
max,8865.360000,293.260000,102.820000,153.070000,164.030000,229.390000,929.000000,64.340000


In [32]:
#If the concentration of one pollutant is high, is it more likely that the concentration of another pollutant is also high?
air_df_full.corr()

,CO,NO,NO2,O3,SO2,PM2.5,PM10,NH3
CO,1.000000,0.801476,0.534978,-0.324413,0.695676,0.598816,0.319395,0.686254
NO,0.801476,1.000000,0.541790,-0.299892,0.804404,0.341667,0.159957,0.377550
NO2,0.534978,0.541790,1.000000,-0.400242,0.677808,0.569111,0.291007,0.394357
O3,-0.324413,-0.299892,-0.400242,1.000000,-0.199275,-0.164452,0.013984,-0.281973
SO2,0.695676,0.804404,0.677808,-0.199275,1.000000,0.401885,0.179974,0.354372
PM2.5,0.598816,0.341667,0.569111,-0.164452,0.401885,1.000000,0.805945,0.573077
PM10,0.319395,0.159957,0.291007,0.013984,0.179974,0.805945,1.000000,0.337422
NH3,0.686254,0.377550,0.394357,-0.281973,0.354372,0.573077,0.337422,1.000000


From the table above, we can see 
There is a Positive correlation between CO and PM2,5; PM2.5 and PM10
                               

In [33]:
air_df_full['CO'].aggregate('mean')

591.1689937106918

In [34]:
#air_df_full.groupby('Continental Region') # groupby function doesnot work

In [35]:
air_df_full['Continental Region'].value_counts()

Asia       61
Africa     51
Europe     39
Oceania     8
Name: Continental Region, dtype: int64

In [36]:
#Contruct data frame for each Continental Region
#Asia
Asia_df = air_df_full.loc[ air_df_full['Continental Region'] == 'Asia' ]
Asia_df = Asia_df.reset_index(drop=True)
#Africa
Africa_df = air_df_full.loc[ air_df_full['Continental Region'] == 'Africa' ]
Africa_df = Africa_df.reset_index(drop=True)
#Europe
Europe_df= air_df_full.loc[ air_df_full['Continental Region'] == 'Europe' ]
Europe_df = Europe_df.reset_index(drop=True)
#Oceania
Oceania_df = air_df_full.loc[ air_df_full['Continental Region'] == 'Oceania' ]
Oceania_df = Oceania_df.reset_index(drop=True)

In [37]:
Asia_df[ ['CO', 'NO', 'NO2', 'O3', 'SO2','PM2.5','PM10','NH3'] ].aggregate('mean')

CO       920.237213
NO        20.699180
NO2       25.688361
O3        39.271803
SO2       20.625410
PM2.5     55.177705
PM10      82.332459
NH3        6.820164
dtype: float64

In [38]:
Africa_df[ ['CO', 'NO', 'NO2', 'O3', 'SO2','PM2.5','PM10','NH3'] ].aggregate('mean')

CO       432.188039
NO         1.496078
NO2        5.057843
O3        30.190784
SO2        2.726863
PM2.5     29.948627
PM10      86.020784
NH3        2.599412
dtype: float64

In [39]:
Europe_df[ ['CO', 'NO', 'NO2', 'O3', 'SO2','PM2.5','PM10','NH3'] ].aggregate('mean')

CO       363.570769
NO         6.556410
NO2       13.055385
O3        29.885641
SO2        3.290513
PM2.5     21.615128
PM10      32.752564
NH3        1.457692
dtype: float64

In [40]:
Oceania_df[ ['CO', 'NO', 'NO2', 'O3', 'SO2','PM2.5','PM10','NH3'] ].aggregate('mean')

CO       205.06875
NO         0.20375
NO2        0.58625
O3        39.51875
SO2        0.52250
PM2.5      2.10750
PM10       4.37875
NH3        0.35875
dtype: float64

In [41]:
#Do you find any relations between continental region and the pollution levels?

The city in Asia normally has higher pollution levels

The city in Oceania normally has Lower pollution levels 